In [1]:
import pandas as pd
import warnings
import numpy as np
import datetime
import calendar as c
import pyarrow

In [2]:
data_file = 'bank_enriched_addresses.csv'
df_raw = pd.read_csv(filepath_or_buffer=data_file, header=0)

# preview data
df_raw.head()

,age,name,address,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,30,giovanni kessler,"17864 Grace Field\nLonzoborough, IN 28057",unemployed,married,primary,no,1787,no,no,cellular,19,oct,79,1,-1,0,unknown,no
1,33,domenico swift,"3887RATKEMEADOWS\NNORTHLEANNATON,VI94092-4948",services,married,secondary,no,4789,yes,yes,cellular,11,may,220,1,339,4,failure,no
2,35,talon gulgowski,"056 pfannerstill cape st.\neast marta, nj 21058",management,single,tertiary,no,1350,yes,no,cellular,16,apr,185,1,330,1,failure,no
3,30,cayla pagac,"146 gay fort suite 622\nwest kathryn, la 81781",management,married,tertiary,no,1476,yes,yes,unknown,3,jun,199,4,-1,0,unknown,no
4,59,sim ratke,"779 Hegmann Harbors St.\nIsaacmouth, MS 13269-...",blue-collar,married,secondary,no,0,yes,no,unknown,5,may,226,1,-1,0,unknown,no


In [3]:
df_raw.describe()

,age,balance,day,duration,campaign,pdays,previous
count,4521.000000,4521.000000,4521.000000,4521.000000,4521.000000,4521.000000,4521.000000
mean,41.170095,1422.657819,15.915284,263.961292,2.793630,39.766645,0.542579
std,10.576211,3009.638142,8.247667,259.856633,3.109807,100.121124,1.693562
min,19.000000,-3313.000000,1.000000,4.000000,1.000000,-1.000000,0.000000
25%,33.000000,69.000000,9.000000,104.000000,1.000000,-1.000000,0.000000
50%,39.000000,444.000000,16.000000,185.000000,2.000000,-1.000000,0.000000
75%,49.000000,1480.000000,21.000000,329.000000,3.000000,-1.000000,0.000000
max,87.000000,71188.000000,31.000000,3025.000000,50.000000,871.000000,25.000000


In [4]:
df_raw.dtypes

age           int64
name         object
address      object
job          object
marital      object
education    object
default      object
balance       int64
housing      object
loan         object
contact      object
day           int64
month        object
duration      int64
campaign      int64
pdays         int64
previous      int64
poutcome     object
y            object
dtype: object

In [5]:
# cleaning functions
def get_nth_element(s, n='first'):
    str_list = s.split()
    
    # consider last name to be the last sequence of alphabetic chars in name string
    max_val = len(str_list)-1
    
    # for multiple middle names, dict values can be added programatically if desired
    name_map = {'first':0, 'last':max_val}
    name = str_list[name_map[n]]
    return name

def first_name(n):
    fn = get_nth_element(n)
    return fn

def last_name(n):
    last = len(n)-1
    ln = get_nth_element(n, 'last')
    return ln

def ranged_list(start,stop,step):
    rlist = np.arange(start,stop,step).tolist()
    return rlist

def yesno_to_bool(s):
    yn_map = {'yes':True, 'no':False} # being explicit will catch data issues
    return yn_map[s]

def mabbrev_to_int(abbr):
    m = [s.lower() for s in c.month_abbr]
    month_map=dict(zip(m,ranged_list(0,13,1)))
    
    month_as_int=month_map[abbr]
    return  month_as_int

def padded_2(int_val):
    padded_int=str(int_val).zfill(2)
    return padded_int

def mabbrev_to_padded(abbr):
    padded_int=padded_2(mabbrev_to_int(abbr))
    return padded_int

## cleaning
1.  Remove all rows where pdays is -1
1.  Split name into first name and second name columns (drop name)
1.  Replace the values in the age column with bucketed values, such that < 10 becomes 0, 10
<= x < 20 becomes 1, etc.
1.  Replace yes/no values with booleans
1.  Replace day and month with a single date column, of the form dd/MM
1.  Rename the y column “outcome"

In [6]:
yesno_cols = ['default','housing','loan','y']
# check yesno col for bad values
for col in yesno_cols:
    print('column name: ', col,'\n',df_raw[col].unique())

column name:  default 
 ['no' 'yes']
column name:  housing 
 ['no' 'yes']
column name:  loan 
 ['no' 'yes']
column name:  y 
 ['no' 'yes']


In [7]:
# avoid mutating df_raw
clean = df_raw.copy()

# we could use drop but that would be inefficient as it slices twice
clean = clean.loc[clean['pdays']!=-1]

clean['first_name']=clean['name'].apply(first_name)
clean['last_name']=clean['name'].apply(last_name)
clean.drop(labels='name', axis=1, inplace=True)

# for ages 0 to 90
labels = ranged_list(0,9,1)
bins = ranged_list(0,100,10)
clean['age_bucket']=pd.cut(clean['age'], bins=bins, right=False, labels=labels)

# convert all yesno col to bool
for col in yesno_cols:
    clean[col]=clean[col].apply(yesno_to_bool)

# pad M/d & join
clean['month_padded']=clean['month'].apply(mabbrev_to_padded)
clean['day_padded']=clean['day'].apply(padded_2)
clean['date']=clean['day_padded'] + '/' + clean['month_padded']

# rename col
clean.rename(columns={'y':'outcome'}, inplace=True)


## add classifying information
add a column which categorizes geographical features in the address, where present.
Note the dirtiness of the address data and that the exact categories :
-  “water”, where the address contains e.g. lake, creek
-  “relief”, where the address contains e.g. hill, canyon
-  “flat”, where the address contains e.g. plain

In [8]:
def get_geo_features():
    water_feature=['lake','creek','stream','ocean','river']
    relief_feature=['hill','canyon','mountain','valley']
    flat_feature=['plain','meadow', 'prairie']
    return water_feature, relief_feature, flat_feature

# flattening function
def all_keywords(keywords):
    kw_list=[]
    for i in keywords:
        kw_list += i
    return kw_list
        

# extract keywords
# splits out a string by whitespace and compares each string to words in a list
def extract_kw(phrase, kw_list):
    words = phrase.lower().split()
#     words = phrase.lower().split(r'\n')[0].split() <= if we only want to look for keywords within the first line
    
    for w in words:
        for k in kw_list:
            if w==k:
#                 print('found keyword match')
                found = k
                return found
                break
            else:
                found = None
    return found

# maps the same category value to several features
# used to categorize discreet words into categories
def get_feature_dict(feature_value, features):
    val_list=[feature_value]*len(features)
    feature_dict = dict(zip(features,val_list))
    return feature_dict


def extract_geo_from_address(address):
#     extract keyword from address
    keyword = extract_kw(address, all_keywords(get_geo_features()))
    
    if keyword==None:
        return None
    
#     generate dictionary for each category
    w,r,f=get_geo_features()
    features=['water','relief','flat']
    dict_list=[get_feature_dict(x,y) for x,y in zip(features,[w,r,f])]
    feature_dict = {k: v for d in dict_list for k, v in d.items()}
    
    category = feature_dict[keyword]
    
    return category

    

##### limitations
- this assumes that all addresses are written with spaces between words and does not work on addresses that have no whitespace
    - we can improve by importing a library to split out words in cases where there isn't any whitespace
- this doesn't handle addresses spelled with leetspeak

In [9]:
# approaching kw search for cases without whitespace
def extract_kw_2(phrase, kw_list):
    p = phrase.encode('unicode-escape').decode().lower()
    for k in kw_list:
        if k in p:
#             print('found keyword match: ', k)
            found = k
            return found
            break
        else:
            found = None
    return found

def extract_geo_from_address_2(address):
#     extract keyword from address
    keyword = extract_kw_2(address, all_keywords(get_geo_features()))
#     print('keyword: ',keyword)
    if keyword==None:
        return None
    
#     generate dictionary for each category
    w,r,f=get_geo_features()
    features=['water','relief','flat']
    dict_list=[get_feature_dict(x,y) for x,y in zip(features,[w,r,f])]
    feature_dict = {k: v for d in dict_list for k, v in d.items()}
    
    category = feature_dict[keyword]
    
    return category

In [10]:
# categorized data
df_cat = clean.copy()

df_cat['geo_feature']=df_cat['address'].apply(extract_geo_from_address_2)

## sorting
Group by the feature (if you created it, or by some other field if not) and filter out any empty
values, sort by the age bucket (or age if you didn’t do the bucketing), and return a row count.

_comments_
- it doesn't make sense to group by the only feature and sort by age -- we can't sort on age if it's not in the grouping -- so I'll be grouping by feature and age bucket to enable sorting

In [11]:
df_cat.loc[:,['address', 'geo_feature']]

,address,geo_feature
1,"3887RATKEMEADOWS\NNORTHLEANNATON,VI94092-4948",flat
2,"056 pfannerstill cape st.\neast marta, nj 21058",None
5,"3547 Treutel Crossing Apt. 567\nReillytown, AS...",None
6,"177CorneliusCampApt.478\nNikolausbury,AA16457-...",None
9,"03353CORKERYPRAIRIE\NWESTLONNIE,AR87918-1164",flat
...,...,...
4492,"7491KILBACKMEADOWAPT.792\NPAGACMOUTH,AS11081-6594",flat
4504,"7081 leffler wall ave.\nkemmerside, dc 00182",None
4514,"25096 William Fi3ld\nMcCull0ughm0uth, 0R 33380...",None
4519,"9234 BORER MEADOW AVE.\NPORT LUNABOROUGH, NJ 3...",flat


In [12]:
df_grouped = df_cat.copy()
df_grouped = df_grouped.loc[df_grouped['geo_feature']!=None]

agg=df_grouped.groupby(['geo_feature','age_bucket']).size()
agg2=df_grouped.groupby(['geo_feature','age_bucket']).size().unstack(fill_value=0)

In [13]:
agg

geo_feature  age_bucket
flat         0              0
             1              0
             2              2
             3              8
             4              8
             5              3
             6              0
             7              2
             8              0
relief       0              0
             1              0
             2              4
             3             10
             4              6
             5              7
             6              0
             7              2
             8              1
water        0              0
             1              0
             2              6
             3             21
             4             12
             5              3
             6              0
             7              1
             8              0
dtype: int64

In [14]:
# write cleaned row data
# write cleaned data to csv
clean.to_csv(path_or_buf='cleaned_data.csv',sep=',')
# write cleaned data to parquet
clean.to_parquet(path='cleaned_data.parquet', compression='snappy')
print('success')

# write agg
# write agg to csv
agg2.to_csv(path_or_buf='aggregate_rc_geofeature+age.csv', sep=',')


success


In [15]:
# flatten multi-index for parquet
col_values = agg2.columns.categories
x = agg2.copy()
x.columns=[str(s) for s in col_values]
# write agg to parquet
x.to_parquet(path='aggregate_rc_geofeature+age.parquet', compression='snappy')